In [ ]:
!pip install  sentence-transformers bertopic -U


In [ ]:
import pandas as pd
import seaborn as sns
from bertopic import BERTopic
# import math 
import numpy as np
pd.set_option("display.precision", 3)
pd.set_option('display.width', 1024)
pd.set_option('display.max_colwidth', None)
%matplotlib inline 

TARGET = "picks_ratio" ## "picks"

model_name = "/content/drive/MyDrive/research/cah/cah_tsdae-model"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Load data

* Note there are some near-duplicate cards (e.g. footballers - could minhash - https://github.com/chrisjmccormick/MinHash/blob/master/runMinHashExample.py , LSH/shingles : https://onestopdataanalysis.com/lsh/  (broken library?) , https://github.com/zyocum/dedup/blob/master/dedup.py ,

`Datasketch` https://www.learndatasci.com/tutorials/building-recommendation-engine-locality-sensitive-hashing-lsh-python/ ,
https://stackoverflow.com/questions/25114338/approximate-string-matching-using-lsh , 

SKLearn friendly?: http://ethen8181.github.io/machine-learning/recsys/content_based/lsh_text.html

`dedupe` library - https://github.com/dedupeio/dedupe

* Paraphrase mining using sentence bert - https://www.sbert.net/examples/applications/paraphrase-mining/README.html  (may be too semantic)

* Example: "10 football players with erections *barreling* towards you at full speed" - 10 football players with erections **barrelling** towards you at full speed."

Task modelling:
* Ranking = most accurate
* Regression/classification: can be done just over single cards, or between pairs. 
    * if pairs-wise: can use sentencebert/SNLI models (which assume pairs of sentences) - https://github.com/UKPLab/sentence-transformers
    
    
Examples:

* https://www.sbert.net/examples/training/sts/README.html#training-data
* https://github.com/UKPLab/sentence-transformers/blob/master/examples/training/nli/training_nli.py
* https://github.com/UKPLab/sentence-transformers/blob/master/examples/training/nli/training_nli_v2.py


* Keras example (not sentenceBert): https://keras.io/examples/nlp/semantic_similarity_with_bert/


* Interpretability tool : https://github.com/cdpierse/transformers-interpret

In [ ]:
df = pd.read_parquet("/content/drive/MyDrive/research/cah/cah_train_min3_games.parquet").drop(columns=["prior_white"],errors="ignore") 
df["won"] = df["won"].astype(int)
df.rename(columns={"won":"picks","sum_won":"sum_picks"},inplace=True)
df

,fake_round_id,black_card_text,white_card_text,picks,text,sum_picks
2989541,298955,Oh my god! __ killed Kenny!,Jeff Bezos,1,Oh my god! Jeff Bezos killed Kenny!,3
2989540,298955,Oh my god! __ killed Kenny!,Brief male nudity,0,Oh my god! Brief male nudity killed Kenny!,1
2989542,298955,Oh my god! __ killed Kenny!,A bird that shits human turds,0,Oh my god! A bird that shits human turds killed Kenny!,0
2989543,298955,Oh my god! __ killed Kenny!,Holding up the line at Walgreens by trying to use an expired coupon,0,Oh my god! Holding up the line at Walgreens by trying to use an expired coupon killed Kenny!,0
2989544,298955,Oh my god! __ killed Kenny!,Powerful thighs,0,Oh my god! Powerful thighs killed Kenny!,0
...,...,...,...,...,...,...
4,1,"Hi MTV! My name is Kendra, I live in Malibu, I'm into __, and I love to have a good time.",A sorry excuse for a father,0,"Hi MTV! My name is Kendra, I live in Malibu, I'm into A sorry excuse for a father, and I love to have a good time.",0
6,1,"Hi MTV! My name is Kendra, I live in Malibu, I'm into __, and I love to have a good time.",A full-on panic attack,0,"Hi MTV! My name is Kendra, I live in Malibu, I'm into A full-on panic attack, and I love to have a good time.",0
7,1,"Hi MTV! My name is Kendra, I live in Malibu, I'm into __, and I love to have a good time.",Chunky highlights,0,"Hi MTV! My name is Kendra, I live in Malibu, I'm into Chunky highlights, and I love to have a good time.",1
8,1,"Hi MTV! My name is Kendra, I live in Malibu, I'm into __, and I love to have a good time.",Your mom,0,"Hi MTV! My name is Kendra, I live in Malibu, I'm into Your mom, and I love to have a good time.",2


In [ ]:
# df = pd.read_csv("cah_lab_data_for_research_dan_ofer.csv")
# df = pd.read_csv("/content/drive/MyDrive/Research/CAH/cah_min6_v2.csv.gz")

display(df.tail())
## alt target - weighted % of times picked - regression target - give more weight to those picked more than 1 time. (log2(1) == 0)
# df["picks_ratio"] = (df["picks_sum"] + math.log2(df["picks_sum"]))/df["pick_opportunities"]
# df["picks_ratio"] = np.where(df['picks_sum']==0, 0, (df["picks_sum"] + np.log2(df["picks_sum"].values))/df["pick_opportunities"]) # ORIG
df = df.sort_values("picks",ascending=False)
## Traing target: % of times picked as funny + upweigh for cases picked more than once
df["picks_ratio"] = df.groupby("text")["picks"].transform("mean")
df["multi_win"] = (df.groupby("text")["picks"].transform("sum")>1).astype(int)
df["picks_ratio"] = (df["picks_ratio"]+df["multi_win"])/2
# df["picks_ratio"] = np.where(df['picks_sum']==0, 0, (df["picks_sum"] + np.log2(df["picks_sum"].values))/df["pick_opportunities"])
# df["picks_ratio"] = df["picks_ratio"].clip(upper=1.)
# print(df.describe())
df = df[["text",'black_card_text', 'white_card_text', 'picks',"picks_ratio"]]

### keep just text ratios:
df = df[["text","picks_ratio","picks"]].drop_duplicates(subset="text").sample(frac=1)
# df["picks"] = df["picks"].astype(float)
print(df.describe())
df

,fake_round_id,black_card_text,white_card_text,picks,text,sum_picks
4,1,"Hi MTV! My name is Kendra, I live in Malibu, I'm into __, and I love to have a good time.",A sorry excuse for a father,0,"Hi MTV! My name is Kendra, I live in Malibu, I'm into A sorry excuse for a father, and I love to have a good time.",0
6,1,"Hi MTV! My name is Kendra, I live in Malibu, I'm into __, and I love to have a good time.",A full-on panic attack,0,"Hi MTV! My name is Kendra, I live in Malibu, I'm into A full-on panic attack, and I love to have a good time.",0
7,1,"Hi MTV! My name is Kendra, I live in Malibu, I'm into __, and I love to have a good time.",Chunky highlights,0,"Hi MTV! My name is Kendra, I live in Malibu, I'm into Chunky highlights, and I love to have a good time.",1
8,1,"Hi MTV! My name is Kendra, I live in Malibu, I'm into __, and I love to have a good time.",Your mom,0,"Hi MTV! My name is Kendra, I live in Malibu, I'm into Your mom, and I love to have a good time.",2
9,1,"Hi MTV! My name is Kendra, I live in Malibu, I'm into __, and I love to have a good time.",Mufasa's death scene,0,"Hi MTV! My name is Kendra, I live in Malibu, I'm into Mufasa's death scene, and I love to have a good time.",0


       picks_ratio       picks
count   452710.000  452710.000
mean         0.087       0.280
std          0.197       0.449
min          0.000       0.000
25%          0.000       0.000
50%          0.000       0.000
75%          0.100       1.000
max          1.000       1.000


,text,picks_ratio,picks
1815347,What gets me wet? Sperm whales,0.250,1
703109,I have solved politics. My solution is A salty surprise.,0.000,0
80801,What killed my boner? A salad for men that's made of metal,0.000,0
1714424,"Money can't buy me love, but it can buy me Firing a rifle into the air while balls deep in a squealing hog.",0.125,1
718885,"Okay here's the pitch. James Franco and Seth Rogen are trying to score some weed, and then Twenty bucks happens.",0.000,0
...,...,...,...
823347,Do the Dew® with our most extreme flavor yet! Get ready for Mountain Dew Little boy penises!,0.000,0
1698291,"After four platinum albums and three Grammys, it's time to get back to my roots, to what inspired me to make music in the first place: Holding up the line at Walgreens by trying to use an expired coupon.",0.100,1
1831862,Girls just wanna have Stalin.,0.500,1
2814600,"Instead of coal, Santa now gives the bad children Establishing dominance.",0.000,0


## Sentence transformers model(s)
* Sentence bert (NLI? semtantic similarity?) 
https://www.sbert.net/docs/training/overview.html
https://www.sbert.net/docs/quickstart.html  , https://github.com/UKPLab/sentence-transformers
* Load data from disk ? 

* list of pretrained sentencebert models: (All are >300 M! slow download) https://docs.google.com/spreadsheets/d/14QplCdTCDwEmTqrn1LH4yrbKvdogK4oQvYO1K1aPR5M/edit#gid=0

* `stsb-distilroberta-base-v2` - 305M sized model - semantic similarity
* `nli-distilroberta-base-v2` - NLI 
* `average_word_embeddings_glove.6B.300d`, `average_word_embeddings_glove.840B.300d` - glove/w2v embeddings

In [ ]:
from sentence_transformers import SentenceTransformer, SentencesDataset, InputExample, losses, evaluation
import torch
print(torch.cuda.is_available())
from torch.utils.data import DataLoader
import torch.utils.data as data_utils
## https://stackoverflow.com/questions/50307707/convert-pandas-dataframe-to-pytorch-tensor
from transformers import AutoTokenizer, AutoModel
## example from : https://www.sbert.net/docs/training/overview.html
from sentence_transformers import SentenceTransformer, SentencesDataset, InputExample, losses 


True


In [ ]:
## pretrained weights - download vis program often fails, easier to download externally from ftp
# model = SentenceTransformer('stsb-distilroberta-base-v2') ## was distilbert-base-nli-mean-tokens
# model = SentenceTransformer('average_word_embeddings_glove.6B.300d') ## was distilbert-base-nli-mean-tokens
# model = SentenceTransformer("nli-distilroberta-base-v2",device="cuda")

# model = SentenceTransformer("paraphrase-MiniLM-L12-v2",device="cuda") ## "paraphrase-MiniLM-L6-v2". "paraphrase-MiniLM-L12-v2"

model = SentenceTransformer(model_name,device="cuda") 
# model = SentenceTransformer(model_name) 

In [ ]:
model.max_seq_length

128

In [ ]:
# len(model.tokenize(["ad and "])["input_ids"])
model.get_sentence_embedding_dimension() # 768

384

#### redo above, but with real data


In [ ]:
TEST_SIZE = df.shape[0]//12 # 37725
print(TEST_SIZE)

In [ ]:
# df["picks"] = df["picks"].astype(int) # for softmax model 
df.iloc[TEST_SIZE:]

In [ ]:
## is list correct? error otherwise? 
# train_examples = list(df.iloc[TEST_SIZE:].apply(lambda row: InputExample(texts=[ row["black_card_text"],row["white_card_text"]], label=row[TARGET]),axis=1))
# test_examples =  list(df.iloc[0:TEST_SIZE].apply(lambda row: InputExample(texts=[row["black_card_text"],row["white_card_text"]], label=row[TARGET]),axis=1))

train_examples = list(df.iloc[TEST_SIZE:].apply(lambda row: InputExample(texts=[ row["text"]], label=row[TARGET]),axis=1))
# test_examples =  list(df.iloc[0:TEST_SIZE].apply(lambda row: InputExample(texts=[row["black_card_text"],row["white_card_text"]], label=row[TARGET]),axis=1))


In [ ]:
# dev_eval = evaluation.BinaryClassificationEvaluator(sentences1=df.iloc[0:2500,0],
#     sentences2= df.iloc[0:2500,1],
#     labels=df.iloc[0:2500,2],show_progress_bar=True,batch_size=256)
dev_eval = evaluation.BinaryClassificationEvaluator(sentences1=list(df.iloc[0:TEST_SIZE,0].values),
    sentences2= list(df.iloc[0:TEST_SIZE,1].values),
    labels=list(df.iloc[0:TEST_SIZE,2].values),show_progress_bar=True,batch_size=128)

In [ ]:
## https://github.com/UKPLab/sentence-transformers/blob/master/examples/training/nli/training_nli.py 

## evaluator example 
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=64)

In [ ]:
%%time
train_loss = losses.CosineSimilarityLoss(model) # ORIG.  # label must be float  - regression 
# train_loss = losses.SoftmaxLoss(model,  # label must be int 
#     sentence_embedding_dimension= model.get_sentence_embedding_dimension(),
#     num_labels= 2,
#     concatenation_sent_rep= True,
#     concatenation_sent_difference = True,
#     concatenation_sent_multiplication = True) # ALT - for classification https://www.sbert.net/docs/package_reference/losses.html#softmaxloss

#Tune the model
model.fit(train_objectives=[(train_dataloader, train_loss)], 
          epochs=1, 
          warmup_steps=(df.shape[0]//64) *2*10, # 100 / or 10% of data?
          evaluator=dev_eval,
          use_amp=True,
          optimizer_params= {'lr': 2e-04}, # 2e-05
          output_path="./output",
        #   save_best_model = True,
         )
### On regression target, min 6, ; Cosine : 62% accuracy, 89 % recall, 47% precision, 62% f1

# BertTopic
* Try to use our (finetuned?) model 

* base on : https://www.kaggle.com/danofer/topic-modeling-arxiv-abstract-with-bertopic

* We can get topics for just the white/answer cards (drop duplicates if so) ;  or the Q&A combination 

* Semi supervised topic model : https://colab.research.google.com/drive/1bxizKzv5vfxJEB29sntU__ZC7PBSIPaQ?usp=sharing#scrollTo=2FNFTvhadAE5 
* Hyperpaams documentation: BERTopic - Advanced Customization.ipynb

In [ ]:
df.shape[0]

452710

In [ ]:
# WHITE_TOPICS_ONLY = False #True

# if WHITE_TOPICS_ONLY:
#  # either white only (problematic if as target) , or the combinations 
#     df2 = df.drop_duplicates(["white_card_text"])
#     # df2 = df.groupby(["white_card_text"]).max().reset_index().drop_duplicates(["white_card_text"]) ## can't do - all the white cards have at least "1" pick globally
#     print(df2.shape)
#     abstracts = list(df2["white_card_text"])
#     targets = list(df2["picks"].astype(int).values) ##df.drop_duplicates(["white_card_text"])["picks"].astype(int).values
# else:## alt : use both cols 
#     abstracts = list(df['black_card_text'] +" " + df['white_card_text'])
#     # targets = df.drop_duplicates(["white_card_text","black_card_text"])["picks"].astype(int).values
#     targets = list(df["picks"].astype(int).values)

### new:

abstracts = list(df["text"].head())
targets = list(df["picks"].values)

In [ ]:
df2 = df.sample(133456).copy()

abstracts = list(df2["text"].copy())
targets = list(df2["picks"].copy())

In [ ]:
%%time
##ORIG: 
# topic_model = BERTopic(verbose=True, embedding_model=model, min_topic_size=10)
### Alt: semi supervised - Warning! problematic if used with white only data (labels will conflict)
topic_model = BERTopic(verbose=True, embedding_model=model, min_topic_size=12, top_n_words=7,
                       n_gram_range=(1,2),nr_topics=100,low_memory=True) ## 1k topics with 100k words by deault/"auto"


CPU times: user 886 µs, sys: 0 ns, total: 886 µs
Wall time: 903 µs


In [ ]:
%%time
topic_model.fit(documents=abstracts,y=targets)

Batches:   0%|          | 0/1046 [00:00<?, ?it/s]

2021-12-08 18:25:50,735 - BERTopic - Transformed documents to Embeddings
2021-12-08 18:26:29,626 - BERTopic - Reduced dimensionality with UMAP
2021-12-08 18:26:35,735 - BERTopic - Clustered UMAP embeddings with HDBSCAN
2021-12-08 18:27:36,154 - BERTopic - Reduced number of topics from 719 to 121


In [ ]:
%%time
# topics, _ = topic_model.fit_transform(abstracts); len(topic_model.get_topic_info()) 
# topics, _ = topic_model.fit_transform(documents=abstracts,y=targets)
topics, _ = topic_model.transform(documents=abstracts)
print(len(topic_model.get_topic_info())) # supervised topic model

Batches:   0%|          | 0/1046 [00:00<?, ?it/s]

121


In [ ]:
topic_model.get_topic_info().head(12)

,Topic,Count,Name
0,-1,8473,-1_re_are_out_we
1,0,1466,0_of the_with the_and the_about the
2,1,1296,1_full of_no_when_richards
3,2,633,2_laid_you laid_and watching_getting naked
4,3,467,3_developed smaller_efficient version_dear brutus_fault
5,4,457,4_bellied boy_pot bellied_bellied_my disgusting
6,5,368,5_dog dying_friends dying_my friends_friends
7,6,366,6_ed absolutely_co ed_what harshing_harshing
8,7,344,7_can handle_bullshit fuck_card again_did it
9,8,331,8_my solution_solved politics_politics_my fettuccine


In [ ]:
topic_model.visualize_barchart(top_n_topics=10, height=700)

For each topic, we have generated 10 words that best represent that topic. However, are 10 words sufficient? Can we already have a good topic representation with 1 word?

To understand the amount of words needed to have a sufficient topic representation, we can show the decline of term score when adding terms. The idea here is that each term that is added has a lower term score than the previous since the first is the best term for the topic. Eventually, we reach the point of diminishing returns, which is very similar to the elbow-method used in k-NN. Below, you can see the decline in term score when adding terms per topic:

In [ ]:
topic_model.visualize_term_rank()

In [ ]:
# topic_model.visualize_term_rank(log_scale=True)

In [ ]:
topic_model.topics

{-1: [('re', 0.0031945780765953264),
  ('are', 0.0031396554934341775),
  ('out', 0.0031146571972092414),
  ('we', 0.0030869973696887676),
  ('not', 0.002992963744714643),
  ('no no', 0.002938465265784058),
  ('our', 0.002778809104169475),
  ('don', 0.0027468744113056612)],
 0: [('of the', 0.008680440585521776),
  ('with the', 0.006144351384461377),
  ('and the', 0.005092465216644301),
  ('about the', 0.004711121591730716),
  ('re the', 0.0035429517242748913),
  ('from the', 0.003470593829649678),
  ('to the', 0.003364011147374374),
  ('baby', 0.00285028864663676)],
 1: [('full of', 0.005625281683444481),
  ('no', 0.0045981960180266224),
  ('when', 0.0036616157279873514),
  ('richards', 0.0034270130962683843),
  ('denise richards', 0.0034270130962683843),
  ('denise', 0.0034270130962683843),
  ('no no', 0.00312880591666346),
  ('out', 0.003087692789755934)],
 2: [('laid', 0.01723548447841761),
  ('you laid', 0.017022730393766457),
  ('and watching', 0.006086026100584151),
  ('getting na

## Topic Relationships
Having extracted the topics and their representations it might be helpful to check out the uniqueness of each topic. Some topics might be quite similar and could be merged or are simply interesting to research further. 

To do this, we start off by mapping our topics to a 2D representation by reducing the topic vectors with UMAP:

In [ ]:
topic_model.visualize_topics(top_n_topics=30)

The distance between topics show, to a certain extend, the similarities between topics. However, to better visualize and understand the similarity between topics, we can use two plots to gives us further insight into that. Namely, visualizing the possible topic hierarchy and its similarity matrix:

In [ ]:
topic_model.visualize_hierarchy(top_n_topics=30, width=800)

In [ ]:
topic_model.visualize_heatmap(n_clusters=20, top_n_topics=80)

## Topics per Class
Lastly, let's focus on the given categories for each paper. Can we find out which topics are frequently found in certain categories? Typically, topic modeling tends to find more topics than the categories that were previously defined. This helps us find and understand certain subcategories that might exist in the data. 

In [ ]:
topics_per_class = topic_model.topics_per_class(abstracts, topics, classes=targets)
# topics_per_class

2it [00:13,  6.73s/it]


,Topic,Words,Frequency,Class
0,-1,"no, re, are, one, out",5379,0
1,0,"of the, and the, about the, re the, from the",1458,0
2,1,"full of, when, denise richards, denise, richards",1195,0
3,2,"getting really, getting naked, crunk, getting crunk, watching nickelodeon",382,0
4,3,"more efficient, smaller more, developed smaller, efficient version, communication communication",343,0
...,...,...,...,...
232,115,"original movie, montana struggles, movie hannah, channel original, hannah montana",25,1
233,116,"burps smell, none unless, hun, want none, anaconda don",20,1
234,117,"entire day, at reasonable, reasonable, to bed, bed at",15,1
235,118,"prescribed, cure for, psychologists, psychologists prescribed, as cure",20,1


In [ ]:
topics_per_class[topics_per_class["Class"]==0].sort_values("Frequency",ascending=False).head(12)

,Topic,Words,Frequency,Class,Name
0,-1,"no, re, are, one, out",5379,0,-1_re_are_out_we
1,0,"of the, and the, about the, re the, from the",1458,0,0_of the_with the_and the_about the
2,1,"full of, when, denise richards, denise, richards",1195,0,1_full of_no_when_richards
5,4,"my disgusting, bellied, bellied boy, pot bellied, disgusting pot",454,0,4_bellied boy_pot bellied_bellied_my dis...
3,2,"getting really, getting naked, crunk, getting crunk, watching nickelodeon",382,0,2_laid_you laid_and watching_getting nak...
4,3,"more efficient, smaller more, developed smaller, efficient version, communication communication",343,0,3_developed smaller_efficient version_de...
7,6,"ed absolutely, co ed, video called, absolutely destroyed, hot smelly",299,0,6_ed absolutely_co ed_what harshing_hars...
10,9,"prime directive, you exposed, have violated, violated the, directive",240,0,9_violated the_directive you_exposed an_...
14,13,"moon people, tall people, jumping, person eat, jumping out",236,0,13_moon people_tall people_black people_...
8,7,"bullshit fuck, again played, oops did, did it, it again",234,0,7_can handle_bullshit fuck_card again_di...


In [ ]:
topics_per_class[topics_per_class["Class"]==1].sort_values("Frequency",ascending=False).head(20)

,Topic,Words,Frequency,Class,Name
120,-1,"no, out, do, are, don",3094,1,-1_re_are_out_we
126,5,"date with, gentlemen have, have date, ll excuse, me gentlemen",293,1,5_dog dying_friends dying_my friends_fri...
123,2,"most problematic, hand we, shake, shake with, not shake",251,1,2_laid_you laid_and watching_getting nak...
153,32,"homosexual agenda, the homosexual, agenda, hamburglar, the hamburglar",202,1,32_the dentist_dentist_the homosexual_ho...
162,41,"pervades all, existence, pervades, basic suffering, the basic",163,1,41_blood of_existence_basic suffering_pe...
180,60,"to frail, dick strapped, frail, frail white, bleached",138,1,60_frail white_frail_to frail_bleached a...
124,3,"brutus, dear brutus, brutus is, our stars, fault dear",124,1,3_developed smaller_efficient version_de...
133,12,"and nursing, nursing it, raccoon, pulling sick, raccoon out",120,1,12_covered raccoon_raccoon_raccoon out_n...
217,98,"in inconceivable, dare speak, lips dare, aloud and, inconceivable",119,1,98_gnaws_speak aloud_inconceivable_unlig...
139,18,"admirable, your persistence, admirable my, prince but, is admirable",114,1,18_is admirable_admirable my_from rowlin...


In [ ]:
topic_model.visualize_topics_per_class(topics_per_class, top_n_topics=11, width=800)